# TASK DESCRIPTION

**Legend:**

Young Alex has a beloved BERT model that he carries everywhere on his trusty flash drive. One day, during an excursion along the River Styx, a few drops of water landed on the precious device, corrupting the model's weights.

Heartbroken, Alex rushed home to fix the neural network. After quick analysis, he discovered only the token embeddings were damaged - the rest of the architecture (attention blocks and heads) remained perfectly intact. Now he needs to restore the model's performance on Sentiment Analysis Task.

**Task:**

You need to fix the broken vectors of the Embeddings matrix of the model so as to improve the quality of the model on the task of text sentiment analysis.

**Restrictions:**

- You can not use any other transformer based pre-trained models and LLMs.

- You can not any additional data

- You can not fine-tune or pre-train model

===

When you make a submit, make a Quick Save of the notebook, otherwise we may reject your solution.

You must solve this task on KAGGLE (YOU CAN'T USE CLOUD.RU)

==========

**Легенда:**

Young Alex имеет любимую модель BERT, которую он везде носит на своей надежной флешке. Однажды, во время экскурсии вдоль реки Стикс, несколько капель воды попало на драгоценное устройство, повредив веса модели.

С разбитым сердцем Алекс поспешил домой, чтобы починить нейронную сеть. После быстрого анализа он обнаружил, что повреждены только эмбеддинги токенов — остальная архитектура (блоки внимания и головы) осталась полностью нетронутой.

Теперь ему нужно восстановить производительность модели, оставив все остальные веса замороженными (никакие изменения в механизмах внимания или других компонентах не допускаются). Ваша задача — помочь Алексу достичь этой цели, не нарушив его ностальгическую привязанность к оригинальной модели.

**Задача:**

Вам необходимо починить сломанные вектора матрицы Embeddings модели так, чтобы улучшить качество модели на задаче анализа тональности текста.

**Ограничения:**

- Вы не можете использовать никакие другие предобученные модели на основе архитектуры Трансформер и LLM.

- Вы не можете использовать никакие дополнительные данные.

- Вы не можете дообучать или предобучать модель.

===

При отправке решения сделайте Quick Save ноутбука, иначе мы можем отклонить ваше решение.

Эту задачу необходимо решить на KAGGLE (ВЫ НЕ МОЖЕТЕ ИСПОЛЬЗОВАТЬ CLOUD.RU)


# DEPENDINGS

In [1]:
import numpy as np
import pandas as pd
import torch
np.random.seed(322)

# LOAD DATASET

In [2]:
val_data_path = "/kaggle/input/neoai-2025-broken-bert/val_dataset.csv"
test_data_path = "/kaggle/input/neoai-2025-broken-bert/test.csv"

val_df = pd.read_csv(val_data_path)
test_df = pd.read_csv(test_data_path)

# LOAD TOKENIZER & MODEL

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Ilseyar-kfu/broken_bert")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):        
        classes = ['neutral', 'positive', 'negative']

        self.tokenizer = tokenizer
        self.texts = df['text'].tolist()
        self.labels = df['labels'].map(lambda x: classes.index(x)).tolist()

    def __getitem__(self, idx):
        res = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=256, return_tensors="pt")
        res['labels'] = self.labels[idx]
        return res

    def __len__(self):
        return len(self.labels)

In [5]:
val_dataset = Dataset(val_df, tokenizer)

In [6]:
texts_2_score = val_df["text"].to_list() + test_df["text"].to_list()

In [7]:
from torch.utils.data import DataLoader

val_loader = DataLoader(val_dataset, 32, True)

# MODEL CHANGES

In [8]:
import random
import os

def set_all_seeds(seed=42):
    # Устанавливаем seed для встроенного генератора Python
    random.seed(seed)
    # Устанавливаем seed для хэш-функции Python (опция для контроля поведения хэшей)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Устанавливаем seed для NumPy
    np.random.seed(seed)

    # Устанавливаем seed для PyTorch
    torch.manual_seed(seed)
    # Устанавливаем seed для генератора на CUDA
    torch.cuda.manual_seed(seed)
    # Отключаем недетерминированное поведение в алгоритмах CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds()

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("Ilseyar-kfu/broken_bert").to('cuda')

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

2025-05-08 14:48:45.518171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746715725.719899      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746715725.775270      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [10]:
def forward_with_custom_embeddings(model, inputs, new_embedings):
    # Сохраняем оригинальные веса
    original_weights = model.bert.embeddings.word_embeddings.weight.clone()

    # Обнуляем все остальные веса
    model.bert.embeddings.word_embeddings = new_embedings

    # Прогоняем модель с измененными весами
    outputs = model(**inputs).logits

    # Восстанавливаем оригинальные веса
    with torch.no_grad():
        model.bert.embeddings.word_embeddings.weight.copy_(original_weights)

    return outputs

In [11]:
# НЕ ОБУЧАЮ МОДЕЛЬ, ОТДЕЛЬНЫЕ ЕМБЕДИНГИ !!!!!!!!!!!!!!!!!!!!!
# Использую модель чисто как помощь в обучении !!!!!!!!!

new_embedings = torch.nn.Embedding(30522, 768, padding_idx=0)
new_embedings.weight = torch.nn.Parameter(model.bert.embeddings.word_embeddings.weight.clone()) # изначальные как у bert
new_embedings = new_embedings.to('cuda')

# MAGIC IS HERE
optimizer = torch.optim.AdamW(new_embedings.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
# Changing embeddings
from sklearn.metrics import accuracy_score
from tqdm import tqdm

num_epochs = 1
for epoch in range(1, num_epochs + 1):
    losses = list()
    accs = list()
    progress_bar = tqdm(val_loader)
    for batch in progress_bar:
        labels = batch.pop('labels').to('cuda')
    
        for key in batch:
            batch[key] = batch[key].squeeze().to('cuda')
        
        optimizer.zero_grad()

        # прогон через ембединги НЕ ТРОГАЯ ЕМБЕДИНГИ МОДЕЛИ
        logits = forward_with_custom_embeddings(model, batch, new_embedings)
        loss = loss_fn(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(new_embedings.parameters(), 5.0)
        optimizer.step()

        losses.append(loss.item())
        accs.append(accuracy_score(labels.cpu().numpy(), logits.argmax(1).cpu().numpy()))
    
        progress_bar.set_postfix(acc=np.mean(accs).round(4), loss=np.mean(losses).round(4))

100%|██████████| 79/79 [00:57<00:00,  1.37it/s, acc=0.506, loss=1.02]


In [13]:
model.bert.embeddings.word_embeddings.weight = torch.nn.Parameter(torch.Tensor(new_embedings.weight))

### Legalize

In [14]:
# import fasttext

# ft_model = fasttext.train_supervised(input="fasttext.txt", epoch=10, dim=768, lr=0.25)

# EVALUATION

In [15]:
from sklearn.metrics import f1_score
from numpy import argmax
from transformers import pipeline
import wandb
wandb.init(mode= "disabled")

In [16]:
from sklearn.metrics import classification_report

def evaluate_on_validation(model, tokenizer, df_val):
    label_2_dict = {'LABEL_0': 'neutral', "LABEL_1" : 'positive', "LABEL_2": 'negative'}
    classifier = pipeline("text-classification", model= model, tokenizer = tokenizer)
    answ = classifier.predict(list(df_val["text"]))
    answ = [label_2_dict[el["label"]] for el in answ]
    
    # print(f1_score(p.label_ids, preds, average='macro'))
    print(classification_report(df_val["labels"], answ))

In [17]:
evaluate_on_validation(model, tokenizer, val_df)

Device set to use cuda:0


              precision    recall  f1-score   support

    negative       0.86      0.89      0.87       935
     neutral       0.78      0.79      0.79       759
    positive       0.92      0.87      0.89       806

    accuracy                           0.85      2500
   macro avg       0.85      0.85      0.85      2500
weighted avg       0.86      0.85      0.85      2500



# MODEL SCORING
When you make a submit, 
1. Make a Quick Save of the notebook, otherwise we may reject your solution! 
2. Add notebook version to the comment for the submit.

===

При отправке решения:

1. Сделайте Quick Save ноутбука, иначе мы можем отклонить ваше решение!
2. Добавьте версию ноутбука в комментарий к отправке.

In [18]:
import hashlib

def create_submission(model, tokenizer, df_test):
    label_2_dict = {'LABEL_0': 'neutral', "LABEL_1": 'positive', "LABEL_2": 'negative'}
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    answ = classifier.predict(list(df_test["text"]))
    answ = [label_2_dict[el["label"]] for el in answ]
    
    df = pd.DataFrame({"labels" : answ, "id": df_test['id']})
    hsh = hashlib.sha256(df.to_csv(index=False).encode('utf-8')).hexdigest()[:8]
    submit_path = f"submit_{hsh}.csv"
    print(f"SUBMIT_NAME: {submit_path}")
    print(df.head(10))
    df.to_csv(submit_path,index=False)

In [19]:
create_submission(model, tokenizer, test_df)

Device set to use cuda:0


SUBMIT_NAME: submit_ea352c64.csv
     labels    id
0  negative  5000
1  positive  5001
2   neutral  5002
3   neutral  5003
4  positive  5004
5  negative  5005
6  negative  5006
7  negative  5007
8  negative  5008
9  positive  5009
